<a href="https://colab.research.google.com/github/TheDataFestAI/Learning_Resources/blob/main/learning_poc/poc1_ml_clv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# POC for CLV (Customer Life Cycle Value)

## Reference:

1. Domain Knowledge:
    1. https://www.analyticsvidhya.com/blog/2020/10/a-definitive-guide-for-predicting-customer-lifetime-value-clv/
    2. https://towardsdatascience.com/data-driven-growth-with-python-part-3-customer-lifetime-value-prediction-6017802f2e0f
    3. https://www.kaggle.com/code/shailaja4247/customer-lifetime-value-prediction

## Step 1: Get the source data from Kaggle

Ref: https://github.com/TheDataFestAI/Learning_Resources/blob/main/learning_poc/download_data_from_kaggle.ipynb

In [ ]:
# install the required packages
!pip install -q opendatasets

In [9]:
# upload the kaggle.json file into google colab
from google.colab import files

kaggle_filename = "kaggle.json"
kaggle_key = files._upload_file(filepath=kaggle_filename)

In [10]:
# download the desired kaggle dataset
import opendatasets as od

kaggle_dataset_url = 'https://www.kaggle.com/datasets/sergeymedvedev/customer_segmentation'
od.download(kaggle_dataset_url)

100%|██████████| 7.20M/7.20M [00:00<00:00, 92.9MB/s]

## Step 1: Load the data into pandas dataframe

### Get the Encoding of the file

In [16]:
# helpful character encoding module
import charset_normalizer

with open('/content/customer_segmentation/customer_segmentation.csv', "rb") as raw_data:
    result = charset_normalizer.detect(raw_data.read())

print("encoding is {}".format(result))

encoding is {'encoding': 'ascii', 'language': 'English', 'confidence': 1.0}


### Loading into dataframe

In [21]:
import pandas as pd

df = pd.read_csv('/content/customer_segmentation/customer_segmentation.csv',
                 encoding="windows-1250")

print(f"dataframe loaded: {df.shape}")

dataframe loaded: (541909, 8)


### fetch few records from the dataframe

In [22]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## Step 2: Data Cleaning inside Dataframe

### Change the datatype from float to int

In [24]:
df["CustomerID"] = pd.apply

TypeError: ignored

In [23]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [ ]:
# from google.colab import userdata
# userdata.get('kaggle_auth')